In [29]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 16)
pd.set_option('display.width', 800)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'

# fp_df_eval_results = 'df_eval_results'
fp_df_eval_sym_freq_results = 'df_eval_sym_freq_results'

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [30]:
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.

###################################
# df_train, df_val, df_test = _2_split_train_val_test(df_close_clean, s_train=1, s_val=0, s_test=0)
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
###################################

len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f'len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test} ')

len_df_train: 1050, len_df_val: 300, len_df_test: 150 


In [31]:
col_add0 = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt']
col_add1 = ['grp(CAGR)_mean',         'grp(CAGR)_std',         'grp(CAGR)_mean/std']
col_add2 = ['grp(CAGR/UI)_mean',      'grp(CAGR/UI)_std',      'grp(CAGR/UI)_mean/std']
col_add3 = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
col_add4 = ['grp(retnStd/UI)_mean',   'grp(retnStd/UI)_std',   'grp(retnStd/UI)_mean/std']
col_add5 = ['SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI']

col_add_total = col_add0 + col_add1 + col_add2 + col_add3 + col_add4 + col_add5
print(f'col_add_total:\n{col_add_total}, total columns: {len(col_add_total)}')

col_add_total:
['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt', 'grp(CAGR)_mean', 'grp(CAGR)_std', 'grp(CAGR)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI'], total columns: 23


In [32]:
# # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results)
# df

In [33]:
# if store_results:  # record results to df
# my_cols = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_CAGR/UI']
# df_eval_results = pickle_load(path_data_dump, fp_df_eval_results)
df_eval_sym_freq_results = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
# df_eval_sym_freq_results

In [34]:
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
store_results = False
# store_results = True

# number of max lookback tuples to create for iloc start_train:end_train:end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms 
# n_samples = 400  
n_samples = 1

# for training, the number of days to lookback from iloc max-lookback end_train
# days_lookbacks = [15, 30, 60, 120]
# days_lookbacks = [30, 60, 120]
# days_lookbacks = [60, 120]
# days_lookbacks = [120]
# days_lookbacks = [60]
# days_lookbacks = [30]
days_lookbacks = [15]
# days_lookbacks = [15, 30]
# days_lookbacks = [15, 30, 60]
# days_lookbacks = [28]
# days_lookbacks = [32]
# days_lookbacks = [31]
# days_lookbacks = [29]
# days_lookbacks = [15, 29]
# days_lookbacks = [15, 31]
# days_lookbacks = [14, 30]
# days_lookbacks = [16, 32]
# days_lookbacks = [14, 28]

days_lookbacks.sort()

# number of days from end_train are used to evaluate effectiveness of the training
# days_eval = 10
# days_eval = 6
days_eval = 5
# days_eval = 4
# days_eval = 3
# days_eval = 2  


# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  #  start index of n_top_syms for evaluation
# syms_start = 1  #  start index of n_top_syms for evaluation

# syms_end = n_top_syms  #  end index of n_top_syms for evaluation
# syms_end = 1  #  end index of n_top_syms for evaluation
# syms_end = 2  #  end index of n_top_syms for evaluation
# syms_end = 3  #  end index of n_top_syms for evaluation
# syms_end = 4  #  end index of n_top_syms for evaluation
# syms_end = 5  #  end index of n_top_syms for evaluation
# syms_end = 6  #  end index of n_top_syms for evaluation
syms_end = 10  #  end index of n_top_syms for evaluation

# freq_cnt = 6  # frequency of symbol in top_set_syms_n_freq 

create a sets of iloc lookback slices (start_train:end_train:end_eval), where  
end_train - start_train = days_lookback  
end_eval - end_train = days_eval  
for example,  
if given:  
 n_samples = 2  
 days_lookbacks = [30, 60, 120]  
 days_eval = 10  
a possible result is:  
 max_lookback_slices:  
 [(150, 270, 280), (5, 125, 135)]  
 where 270-150=125-5=max(days_lookbacks), 280-270=135-125=days_eval  
 sets_lookback_slices:  
 [[(240, 270, 280), (210, 270, 280), (150, 270, 280)], [(95, 125, 135), (65, 125, 135), (5, 125, 135)]]  
  where in a set, 270-240=days_lookbacks[0], 270-210=days_lookbacks[1], 270-150=days_lookbacks[2]  
  and 270, i.e. end_train, is constant for the set  

In [35]:
# return n_samples slices
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(206, 221, 226)]
sets_lookback_slices:
[[(206, 221, 226)]]


Generate lists, n_samples long, of the highest performance ranked symbols. The performance metrics are: CAGR/UI, CAGR/retnStd, retnStd/UI. n_top_syms of the best performing symbols from each metric are combined. The symbols are sorted by their number of appearances in the combined pool, and are placed in a list. A slice of the best performing symbols is selected by syms_start:syms_end, i.e. top_set_syms_n_freq[syms_start:syms_end].     

The performance metrics are calculated based on slices in sets_lookback_slices.  The first two numbers are ilocs for training. The last two numbers are ilocs for evaluation.

In [36]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    d_eval = end_eval - start_eval

    if verbose:
      print(f'days lookback: {lookback}')
      print(f'lb_slices:     {lb_slices}')
      print(f'lb_slice:      {lb_slice}')
      print(f'days eval:     {d_eval}')    
      print(f'start_train:   {start_train}')
      print(f'end_train:     {end_train}')
      # print(f'start_eval:    {start_eval}')
      # print(f'end_eval:      {end_eval}')`

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=n_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

days lookback: 15
lb_slices:     [(206, 221, 226)]
lb_slice:      (206, 221, 226)
days eval:     5
start_train:   206
end_train:     221
perf_ranks: {'period-15': {'r_CAGR/UI': array(['FLR', 'CTSH', 'NOW', 'ARWR', 'NVAX', 'CORT', 'CRSP', 'STX',
       'ZBRA', 'FATE', 'PYPL', 'STLA', 'PTC', 'COLL', 'ADX', 'TFC',
       'ARES', 'NFLX', 'CSGP', 'BLK'], dtype=object), 'r_CAGR/retnStd': array(['ARWR', 'NVAX', 'CRSP', 'CORT', 'FATE', 'COLL', 'PTCT', 'STLA',
       'STX', 'ICHR', 'PTC', 'ARES', 'RCKT', 'ZBRA', 'ALGN', 'NFLX',
       'ANF', 'KSS', 'NTRA', 'FLR'], dtype=object), 'r_retnStd/UI': array(['FLR', 'CTSH', 'NOW', 'PYPL', 'ADX', 'TFC', 'ZBRA', 'STX', 'KAR',
       'BKI', 'CORT', 'NVO', 'PNC', 'CSQ', 'TRV', 'EOS', 'TYL', 'CSGP',
       'GOOG', 'V'], dtype=object)}}
most_common_syms: [('FLR', 3), ('CORT', 3), ('STX', 3), ('ZBRA', 3), ('CTSH', 2), ('NOW', 2), ('ARWR', 2), ('NVAX', 2), ('CRSP', 2), ('FATE', 2), ('PYPL', 2), ('STLA', 2), ('PTC', 2), ('COLL', 2), ('ADX', 2), ('TFC', 2), ('AR

In [37]:
def top_set_sym_freq_cnt(top_set_syms_n_freq):
    sym_freq_cnt_6 = []
    sym_freq_cnt_5 = []
    sym_freq_cnt_4 = []
    sym_freq_cnt_3 = []
    sym_freq_cnt_2 = []

    for sym_n_freq in top_set_syms_n_freq:
        _sym = sym_n_freq[0]
        _freq = sym_n_freq[1]
        # print(_sym, _freq)
        if _freq == 6:
            sym_freq_cnt_6.append(_sym)
        elif _freq == 5:
            sym_freq_cnt_5.append(_sym)
        elif _freq == 4:
            sym_freq_cnt_4.append(_sym)
        elif _freq == 3:
            sym_freq_cnt_3.append(_sym)          
        else:
            sym_freq_cnt_2.append(_sym)

    l_sym_freq_cnt = []
    l_sym_freq_cnt.append(sym_freq_cnt_6)
    l_sym_freq_cnt.append(sym_freq_cnt_5)
    l_sym_freq_cnt.append(sym_freq_cnt_4)
    l_sym_freq_cnt.append(sym_freq_cnt_3)    
    l_sym_freq_cnt.append(sym_freq_cnt_2)    

    # if verbose:
    #     print(f'sym_freq_cnt_6: {sym_freq_cnt_6}')
    #     print(f'sym_freq_cnt_5: {sym_freq_cnt_5}')
    #     print(f'sym_freq_cnt_4: {sym_freq_cnt_4}')
    #     print(f'sym_freq_cnt_3: {sym_freq_cnt_3}')
    #     print(f'sym_freq_cnt_2: {sym_freq_cnt_2}')

    # return sym_freq_cnt_6, sym_freq_cnt_5, sym_freq_cnt_4, sym_freq_cnt_3, sym_freq_cnt_2
    return l_sym_freq_cnt    

In [38]:
for top_set_syms_n_freq in grp_top_set_syms_n_freq:
  l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
  print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
  print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
  print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
  print(f'sym_freq_3: {l_sym_freq_cnt[3]}')
  print(f'sym_freq_2: {l_sym_freq_cnt[4]}\n')  

sym_freq_6: []
sym_freq_5: []
sym_freq_4: []
sym_freq_3: ['CORT', 'FLR', 'STX', 'ZBRA']
sym_freq_2: ['ADX', 'ARES', 'ARWR', 'COLL', 'CRSP', 'CSGP', 'CTSH', 'FATE', 'NFLX', 'NOW', 'NVAX', 'PTC', 'PYPL', 'STLA', 'TFC', 'ALGN']



In [39]:
# from yf_utils import _7_perf_eval_v1
# from trash_eval import _7_perf_eval_v2
from myUtils import symb_perf_stats_vectorized_v8

print('z_grp_top_set_syms_n_freq:')
z_grp_top_set_syms_n_freq = zip(max_lookback_slices, grp_top_set_syms_n_freq)

for i, (_lookback_slice, _top_set_syms_n_freq) in enumerate(z_grp_top_set_syms_n_freq):
  start_train = _lookback_slice[0]
  end_train = _lookback_slice[1]
  start_eval = end_train
  end_eval = _lookback_slice[2]

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  d_start_train = df_train.index[start_train].strftime('%Y-%m-%d')
  d_end_train = df_train.index[end_train].strftime('%Y-%m-%d')
  d_start_eval = d_end_train
  d_end_eval = df_train.index[end_eval].strftime('%Y-%m-%d')
  d_df_eval_start = d_end_train
  d_df_eval_end = df_train.index[end_eval - 1].strftime('%Y-%m-%d')  
  print(f'max lookback dates: {d_start_train}, {d_end_train}, {d_end_eval}')
  print(f'df_eval dates (inclusive): {d_df_eval_start} - {d_df_eval_end}')    
  print(f'top_set_syms_n_freq: {_top_set_syms_n_freq}\n')

  l_sym_freq_cnt = top_set_sym_freq_cnt(_top_set_syms_n_freq)
  print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
  print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
  print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
  print(f'sym_freq_3: {l_sym_freq_cnt[3]}\n')

  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]


  # _df_perf_SPY, SPY_retnStd_d_UI, SPY_CAGR_d_retnStd, SPY_CAGR_d_UI, SPY_CAGR = _7_perf_eval_v1(df_SPY)
  # _df_perf_SPY, SPY_retnStd_d_UI, SPY_CAGR_d_retnStd, SPY_CAGR_d_UI, SPY_CAGR = _7_perf_eval_v2(df_SPY)  
  # SPY_retnStd_d_UI, SPY_CAGR_d_retnStd, SPY_CAGR_d_UI, SPY_CAGR = _7_perf_eval_v2(df_SPY)  
  # SPY_retnStd_d_UI, SPY_CAGR_d_retnStd, SPY_CAGR_d_UI, SPY_CAGR = symb_perf_stats_vectorized_v8(df_SPY)  
  (
    _symbols,
    _period_yr,
    _retn,
    _DD,
    _UI,
    _MDD,
    _retnMean,
    _retnStd,
    _retnStd_div_UI,
    _CAGR,
    _CAGR_div_retnStd,
    _CAGR_div_UI,
    SPY_retnStd_d_UI,     
    SPY_CAGR,
    SPY_CAGR_d_retnStd,
    SPY_CAGR_d_UI,
  ) = symb_perf_stats_vectorized_v8(df_SPY)  

  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR: {SPY_retnStd_d_UI[0]:>17,.1f}, {SPY_CAGR_d_retnStd[0]:>17,.1f}, {SPY_CAGR_d_UI[0]:>17.1f}, {SPY_CAGR[0]:>17.1f}')

  # drop last list, with frequency count 2 or less, in l_sym_freq_cnt from zip
  zip_cnt_n_syms = zip([6,5,4,3], l_sym_freq_cnt[:-1])  
  for item in zip_cnt_n_syms:
    sym_freq_cnt = item[0]
    syms = item[1]
    if syms:  # iterate ONLY if there are symbols in syms
      df_eval = df_train[start_eval:end_eval][syms]      

      if verbose:
        # print(f'start_eval: {start_eval}')
        # print(f'end_eval:   {end_eval}')  
        print(f'start_eval: {start_eval},  date: {d_end_train}')
        print(f'end_eval:   {end_eval},  date: {d_end_eval},  df_eval last date: {d_df_eval_end}')
        print(f'frequency count of symbol(s): {sym_freq_cnt}')      
        # print(f'\ndf_eval:\n{df_eval}\n')

        syms_n_SPY = syms + ['SPY']
        df_eval_n_SPY = df_train[start_eval:end_eval][syms_n_SPY]   
        print(f'\ndf_eval_n_SPY:\n{df_eval_n_SPY}\n')


      # _, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = _7_perf_eval(df_eval)
      # _df_perf, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI, grp_CAGR = _7_perf_eval_v1(df_eval)
      # _df_perf, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI, grp_CAGR = _7_perf_eval_v2(df_eval)
      # grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI, grp_CAGR = _7_perf_eval_v2(df_eval)
      # grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI, grp_CAGR = symb_perf_stats_vectorized_v8(df_eval)
      (
        _symbols,
        _period_yr,
        _retn,
        _DD,
        _UI,
        _MDD,
        _retnMean,
        _retnStd,
        _retnStd_div_UI,
        _CAGR,
        _CAGR_div_retnStd,
        _CAGR_div_UI,
        grp_retnStd_d_UI,     
        grp_CAGR,
        grp_CAGR_d_retnStd,
        grp_CAGR_d_UI,
      ) = symb_perf_stats_vectorized_v8(df_eval)  
      # print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]  :>13,.3f}, {grp_retnStd_d_UI[1]  :>13,.3f}, {grp_retnStd_d_UI[2]  :>13,.3f}')
      # print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_retnStd[1]:>13,.3f}, {grp_CAGR_d_retnStd[2]:>13,.3f}')
      # print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]     :>13,.3f}, {grp_CAGR_d_UI[1]     :>13,.3f}, {grp_CAGR_d_UI[2]     :>13,.3f}')
      # print(f'grp(CAGR):         mean, std, mean/std: {grp_CAGR[0]          :>13,.3f}, {grp_CAGR[1]          :>13,.3f}, {grp_CAGR[2]          :>13,.3f}')
      print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]  :>17,.1f}, {grp_retnStd_d_UI[1]  :>17,.1f}, {grp_retnStd_d_UI[2]  :>17,.1f}')
      print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>17,.1f}, {grp_CAGR_d_retnStd[1]:>17,.1f}, {grp_CAGR_d_retnStd[2]:>17,.1f}')
      print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]     :>17,.1f}, {grp_CAGR_d_UI[1]     :>17,.1f}, {grp_CAGR_d_UI[2]     :>17,.1f}')
      print(f'grp(CAGR):         mean, std, mean/std: {grp_CAGR[0]          :>17,.1f}, {grp_CAGR[1]          :>17,.1f}, {grp_CAGR[2]          :>17,.1f}')


      if store_results:  # record results to df
        row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end, sym_freq_cnt]
        row_add1      = [grp_CAGR[0],           grp_CAGR[1],           grp_CAGR[2]]
        row_add2      = [grp_CAGR_d_UI[0],      grp_CAGR_d_UI[1],      grp_CAGR_d_UI[2]]
        row_add3      = [grp_CAGR_d_retnStd[0], grp_CAGR_d_retnStd[1], grp_CAGR_d_retnStd[2]]
        row_add4      = [grp_retnStd_d_UI[0],   grp_retnStd_d_UI[1],   grp_retnStd_d_UI[2]]
        row_add5      = [SPY_CAGR[0], SPY_CAGR_d_UI[0], SPY_CAGR_d_retnStd[0], SPY_retnStd_d_UI[0]]
        row_add_total = row_add0 + row_add1 + row_add2 + row_add3 + row_add4 + row_add5
        print(f'row_add_total: {row_add_total}')
        df_eval_sym_freq_results.loc[len(df_eval_sym_freq_results)] = row_add_total
        # print(f'appended row_add to df_eval_results:\n{row_add}\n')
        print(f'appended row_add to df_eval_sym_freq_results:\n{row_add_total}\n')
  
  print('='*50, '\n')  

z_grp_top_set_syms_n_freq:
1 of 1 max_lookback_slice
max_lookback_slice: (206, 221, 226)
max lookback dates: 2018-01-02, 2018-01-24, 2018-01-31
df_eval dates (inclusive): 2018-01-24 - 2018-01-30
top_set_syms_n_freq: [('CORT', 3), ('FLR', 3), ('STX', 3), ('ZBRA', 3), ('ADX', 2), ('ARES', 2), ('ARWR', 2), ('COLL', 2), ('CRSP', 2), ('CSGP', 2), ('CTSH', 2), ('FATE', 2), ('NFLX', 2), ('NOW', 2), ('NVAX', 2), ('PTC', 2), ('PYPL', 2), ('STLA', 2), ('TFC', 2), ('ALGN', 1)]

sym_freq_6: []
sym_freq_5: []
sym_freq_4: []
sym_freq_3: ['CORT', 'FLR', 'STX', 'ZBRA']


SPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR:               1.0,             -32.6,             -33.6,              -0.3
start_eval: 221,  date: 2018-01-24
end_eval:   226,  date: 2018-01-31,  df_eval last date: 2018-01-30
frequency count of symbol(s): 3

df_eval_n_SPY:
                 CORT        FLR        STX        ZBRA         SPY
Date                                                               
2018-01-24  25.080000  57.85339

In [40]:
df_eval_sym_freq_results.columns

Index(['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt', 'grp(CAGR)_mean', 'grp(CAGR)_std', 'grp(CAGR)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI'], dtype='object')

In [41]:
df = df_eval.copy()
df

,CORT,FLR,STX,ZBRA
Date,,,,
2018-01-24,25.080000,57.853394,44.648121,123.349998
2018-01-25,25.059999,57.796753,44.816700,123.500000
2018-01-26,25.100000,58.089413,46.308613,123.480003
2018-01-29,25.500000,58.164944,46.451916,124.610001
2018-01-30,23.680000,56.994274,46.342335,124.129997


In [42]:
df_SPY = df_SPY.copy()
df_SPY

,SPY
Date,
2018-01-24,265.992584
2018-01-25,266.105316
2018-01-26,269.186279
2018-01-29,267.401581
2018-01-30,264.658783


In [43]:
from myUtils import symb_perf_stats_vectorized_v9_
(
  symbols,
  period_yr,
  df_retn,
  df_drawdown,
  arr_UI,
  arr_max_drawdown,
  arr_returns_mean,    
  arr_returns_std,
  arr_returns_std_div_arr_UI,
  arr_CAGR,
  arr_CAGR_div_arr_retnStd,
  arr_CAGR_div_arr_UI,
  grp_returns_std_div_UI,    
  grp_CAGR,
  grp_CAGR_div_retnStd,
  grp_CAGR_div_UI, 
) = symb_perf_stats_vectorized_v9_(df)

print(f'df:\n{df}\n')
print(f'symbols:\n{symbols}\n')
print(f'period_yr:\n{period_yr}\n')
print(f'df_retn:\n{df_retn}\n')
print(f'df_drawdown:\n{df_drawdown}\n')
print(f'arr_UI:\n{arr_UI}\n')
print(f'arr_max_drawdown:\n{arr_max_drawdown}\n')
print(f'arr_returns_mean:\n{arr_returns_mean}\n')
print(f'arr_returns_std:\n{arr_returns_std}\n')
print(f'arr_returns_std_div_arr_UI:\n{arr_returns_std_div_arr_UI}\n')
print(f'arr_CAGR:\n{arr_CAGR}\n')
print(f'arr_CAGR_div_arr_retnStd:\n{arr_CAGR_div_arr_retnStd}\n')
print(f'arr_CAGR_div_arr_UI:\n{arr_CAGR_div_arr_UI}\n')
print(f'grp_returns_std_div_UI:\n{grp_returns_std_div_UI}\n')
print(f'grp_CAGR:\n{grp_CAGR}\n')
print(f'grp_CAGR_div_retnStd:\n{grp_CAGR_div_retnStd}\n')
print(f'grp_CAGR_div_UI:\n{grp_CAGR_div_UI}\n')

df:
                 CORT        FLR        STX        ZBRA
Date                                                   
2018-01-24  25.080000  57.853394  44.648121  123.349998
2018-01-25  25.059999  57.796753  44.816700  123.500000
2018-01-26  25.100000  58.089413  46.308613  123.480003
2018-01-29  25.500000  58.164944  46.451916  124.610001
2018-01-30  23.680000  56.994274  46.342335  124.129997

symbols:
Index(['CORT', 'FLR', 'STX', 'ZBRA'], dtype='object')

period_yr:
0.015873015873015872

df_retn:
                CORT       FLR       STX      ZBRA
Date                                              
2018-01-24  0.000000  0.000000  0.000000  0.000000
2018-01-25 -0.000797 -0.000979  0.003776  0.001216
2018-01-26  0.001596  0.005064  0.033289 -0.000162
2018-01-29  0.015936  0.001300  0.003095  0.009151
2018-01-30 -0.071373 -0.020127 -0.002359 -0.003852

df_drawdown:
                CORT       FLR       STX      ZBRA
Date                                              
2018-01-24  0.000000  0.

In [44]:
from myUtils import symb_perf_stats_vectorized_v8
if verbose:
  _my_df = [df_eval, df_SPY]
  for _df in _my_df:
    (
      symbols,
      period_yr,
      retn,
      DD,
      UI,
      MDD,
      retnMean,
      retnStd,
      retnStd_div_UI,
      CAGR,
      CAGR_div_retnStd,
      CAGR_div_UI,
      grp_retnStd_div_UI,     
      grp_CAGR,
      grp_CAGR_div_retnStd,
      grp_CAGR_div_UI,
    ) = symb_perf_stats_vectorized_v8(df_SPY)

    print(f'_df:\n{_df}\n')
    print(f'symbols:\n{symbols}\n')
    print(f'period_yr:\n{period_yr}\n')
    print(f'retn:\n{retn}\n')
    print(f'DD:\n{DD}\n')
    print(f'UI:\n{UI}\n')
    print(f'MDD:\n{MDD}\n')
    print(f'retnMean:\n{retnMean}\n')
    print(f'retnStd:\n{retnStd}\n')
    print(f'retnStd_div_UI:\n{retnStd_div_UI}\n')
    print(f'CAGR:\n{CAGR}\n')
    print(f'CAGR_div_retnStd:\n{CAGR_div_retnStd}\n')
    print(f'CAGR_div_UI:\n{CAGR_div_UI}\n')
    print(f'grp_retnStd_div_UI:\n{grp_retnStd_div_UI}\n')
    print(f'grp_CAGR:\n{grp_CAGR}\n')
    print(f'grp_CAGR_div_retnStd:\n{grp_CAGR_div_retnStd}\n')
    print(f'grp_CAGR_div_UI:\n{grp_CAGR_div_UI}\n')            

_df:
                 CORT        FLR        STX        ZBRA
Date                                                   
2018-01-24  25.080000  57.853394  44.648121  123.349998
2018-01-25  25.059999  57.796753  44.816700  123.500000
2018-01-26  25.100000  58.089413  46.308613  123.480003
2018-01-29  25.500000  58.164944  46.451916  124.610001
2018-01-30  23.680000  56.994274  46.342335  124.129997

symbols:
Index(['SPY'], dtype='object')

period_yr:
0.015873015873015872

retn:
                 SPY
Date                
2018-01-24  0.000000
2018-01-25  0.000424
2018-01-26  0.011578
2018-01-29 -0.006630
2018-01-30 -0.010257

DD:
                 SPY
Date                
2018-01-24  0.000000
2018-01-25  0.000000
2018-01-26  0.000000
2018-01-29 -0.006630
2018-01-30 -0.016819

UI:
SPY    0.008085
dtype: float64

MDD:
SPY   -0.016819
dtype: float64

retnMean:
SPY   -0.001221
dtype: float64

retnStd:
SPY    0.008328
dtype: float64

retnStd_div_UI:
SPY    1.03006
dtype: float64

CAGR:
SPY   -0.2714

In [45]:
# # Creating Empty DataFrame and Storing it in variable df
# df_eval_results = pd.DataFrame(columns=col_add_total)
pickle_dump(df_eval_sym_freq_results, path_data_dump, fp_df_eval_sym_freq_results)

In [46]:
df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
df

,n_samples,days_lookbacks,days_eval,n_top_syms,syms_start,syms_end,sym_freq_cnt,grp(CAGR)_mean,grp(CAGR)_std,grp(CAGR)_mean/std,grp(CAGR/UI)_mean,grp(CAGR/UI)_std,grp(CAGR/UI)_mean/std,grp(CAGR/retnStd)_mean,grp(CAGR/retnStd)_std,grp(CAGR/retnStd)_mean/std,grp(retnStd/UI)_mean,grp(retnStd/UI)_std,grp(retnStd/UI)_mean/std,SPY_CAGR,SPY_CAGR/UI,SPY_CAGR/retnStd,SPY_retnStd/UI
0,400,"[15, 30, 60,...",5,20,0,10,6,-0.825763,0.217227,-3.801378,-2.962008e+01,1.727265e+01,-1.714855,-113.041403,1.190032e+02,-0.949902,0.538936,0.538472,1.000862,0.117471,1.733638e+02,103.044703,1.682413
1,400,"[15, 30, 60,...",5,20,0,10,5,1.746418,3.760947,0.464356,2.662196e+02,5.002969e+02,0.532123,56.827766,1.646009e+02,0.345246,1.709725,1.091378,1.566575,0.117471,1.733638e+02,103.044703,1.682413
2,400,"[15, 30, 60,...",5,20,0,10,4,-0.553791,0.490615,-1.128769,-2.141196e+01,5.486226e+01,-0.390286,-66.271969,6.625339e+01,-1.000280,0.830215,0.607602,1.366378,0.117471,1.733638e+02,103.044703,1.682413
3,400,"[15, 30, 60,...",5,20,0,10,3,-0.142209,1.209602,-0.117567,1.544800e+02,2.413377e+02,0.640099,54.259532,1.299870e+02,0.417423,1.031155,0.765629,1.346807,0.117471,1.733638e+02,103.044703,1.682413
4,400,"[15, 30, 60,...",5,20,0,10,6,94.677660,164.422328,0.575820,1.877643e+04,3.242281e+04,0.579112,3225.712880,5.557814e+03,0.580392,2.119866,2.181731,0.971644,3.021975,6.757341e+06,1002.103249,6743.158850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14677,400,"[60, 120]",5,20,0,10,3,149.789178,207.661556,0.721314,1.342990e+04,1.855169e+04,0.723918,4066.078489,5.511199e+03,0.737785,2.218532,0.815087,2.721835,11.402758,2.792625e+03,710.872844,3.928445
14678,400,"[60, 120]",5,20,0,10,6,0.032029,0.000000,inf,7.161886e+04,0.000000e+00,inf,443.391866,0.000000e+00,inf,161.524968,0.000000,inf,-0.674979,-6.079998e+01,-233.372205,0.260528
14679,400,"[60, 120]",5,20,0,10,5,13551.034286,23460.885389,0.577601,3.029146e+10,5.246566e+10,0.577358,846993.769709,1.466572e+06,0.577533,9679.607208,15112.073754,0.640521,-0.674979,-6.079998e+01,-233.372205,0.260528
14680,400,"[60, 120]",5,20,0,10,4,33.414005,34.317583,0.973670,6.266861e+03,6.291728e+03,0.996048,1970.848590,2.015399e+03,0.977895,1.854334,1.296145,1.430654,-0.674979,-6.079998e+01,-233.372205,0.260528


In [47]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/

In [48]:
# print(f'len(df.columns): {len(df.columns)}')
# print(f'df.columns: {df.columns}')

In [49]:
# pd.crosstab(df['days_lookbacks'], df['grp(CAGR/UI)_mean'])
tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
        .agg({'grp(CAGR)_mean':          ['count', 'mean', 'std',],
              'grp(CAGR/UI)_mean':       ['mean', 'std',],
              'grp(CAGR/retnStd)_mean':  ['mean', 'std',],
              'SPY_CAGR':                ['mean', 'std',],
              'SPY_CAGR/UI':             ['mean', 'std',],
              'SPY_CAGR/retnStd':        ['mean', 'std',],                           
              })

In [50]:
# tbl

In [51]:
tbl['dif_CAGR_mean'] = tbl['grp(CAGR)_mean', 'mean'] - tbl['SPY_CAGR', 'mean']
tbl['dif_CAGR/UI_mean'] = tbl['grp(CAGR/UI)_mean', 'mean'] - tbl['SPY_CAGR/UI', 'mean'] 
tbl['dif_CAGR/retnStd_mean'] = tbl['grp(CAGR/retnStd)_mean', 'mean'] - tbl['SPY_CAGR/retnStd', 'mean'] 
tbl['grp_CAGR/UI_mean/std'] = tbl['grp(CAGR/UI)_mean', 'mean'] / tbl['grp(CAGR/UI)_mean', 'std']  
tbl['SPY_CAGR/UI_mean/std'] = tbl['SPY_CAGR/UI', 'mean'] / tbl['SPY_CAGR/UI', 'std'] 
tbl['dif_CAGR/UI_mean/std'] = tbl['grp_CAGR/UI_mean/std']  - tbl['SPY_CAGR/UI_mean/std'] 
# tbl.sort_values(by='dif_CAGR/UI_mean', ascending=False, inplace=True)
tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
tbl

grp(CAGR)_mean                             grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                 SPY_CAGR               SPY_CAGR/UI               SPY_CAGR/retnStd              dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std
                                                 count          mean           std              mean           std                   mean           std       mean         std          mean           std             mean          std                                                                                                                    
days_lookbacks   days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                     
[60, 120]        5         3                       800  1.334499e+54  2.176499e+55     2.984031e+60   4.866800e+61     3.041870e+54        4.961133e+55  12.979992  172.950289  8.746892e+06  1.484039e+08       566.524157  4494.014079  1.334499e+54     2.984031e+60     3.041870e+54              0.061314             0.058940             0.002374    
[30]             5         3                       400  1.068806e+12  2.130090e+13     3.384454e+13   6.072343e+14     3.738749e+12        7.443576e+13   5.904440   67.942904  8.998676e+06  1.486335e+08       482.600540  2923.292187  1.068806e+12     3.384453e+13     3.738749e+12              0.055736             0.060543            -0.004807    
[32]             5         3                       400  2.995721e+06  3.697529e+07     2.591098e+12   5.080106e+13     3.177730e+07        3.885437e+08  16.171603  220.266993  8.360448e+06  1.483299e+08       668.723154  5496.787226  2.995705e+06     2.591090e+12     3.177663e+07              0.051005             0.056364            -0.005359    
[60, 120]        5         5                       707  8.434667e+36  1.584728e+38     4.213641e+37   7.916705e+38     1.272389e+37        2.390599e+38  14.547238  183.927649  9.761665e+06  1.578411e+08       616.436004  4775.306791  8.434667e+36     4.213641e+37     1.272389e+37              0.053225             0.061845            -0.008620    
[15]             5         3                       800  6.111776e+24  1.728671e+26     2.576346e+26   7.286443e+27     7.254696e+24        2.051938e+26   4.120580   48.579026  4.921775e+06  1.051586e+08       377.260224  2134.874207  6.111776e+24     2.576346e+26     7.254696e+24              0.035358             0.046803            -0.011445    
[60, 120]        5         6                       596  7.805915e+19  1.597085e+21     1.745456e+26   3.571190e+27     3.250182e+20        6.683117e+21  17.201273  200.240320  1.147523e+07  1.718756e+08       700.061778  5193.713718  7.805915e+19     1.745456e+26     3.250182e+20              0.048876             0.066765            -0.017889    
[15, 30]         6         5                       366  9.875176e+09  1.887534e+11     8.650647e+11   9.644124e+12     3.754284e+10        7.173448e+11  13.028058  168.280075  2.451990e+05  2.181298e+06       496.286139  4195.913218  9.875176e+09     8.650645e+11     3.754284e+10              0.089699             0.112410            -0.022711    
                           3                       400  7.833217e+05  7.922723e+06     1.870569e+12   1.940509e+13     2.392484e+07        2.587860e+08  11.946625  160.990103  2.602247e+05  2.179393e+06       461.938001  4015.128213  7.833097e+05     1.870569e+12     2.392438e+07              0.096396             0.119402            -0.023007    
[60, 120]        5         4                       785  3.564465e+20  9.852566e+21     7.872627e+26   2.202965e+28     7.153491e+21        2.000188e+23  13.198458  174.589341  8.914012e+06  1.498119e+0

In [52]:
# # https://stackoverflow.com/questions/31674195/plot-normal-distribution-given-mean-and-sigma-python
# # loc is mean, scale is standard deviation
# import pylab
# import numpy as np
# from scipy.stats import norm
# # x = np.linspace(-10000,100000,1000)
# x = np.linspace(-40e+10,50e+10,1000)
# y = norm.pdf(x, loc=2.562777e+10, scale=1.036925e+11)    # loc = mean, scale = standard deviation
# # z = norm.pdf(x, loc=3.540615e+10, scale=1.194430e+11)    # for example
# # z1 = norm.pdf(x, loc=298.805901, scale=826.875749)    # for example
# # z1 = norm.pdf(x, loc=1.021825, scale=1.505096)    # for example
# pylab.plot(x,y, 'b')
# # pylab.plot(x,z, 'g')
# # pylab.plot(x,z1, 'r')
# pylab.show()

In [53]:
# # get values of _cols, where grp(CAGR/retnStd)_mean is max after filtering out inf
# _cols = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
# # _df_no_inf = df.loc[df['grp(CAGR/retnStd)_mean'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# # _idx = _df_no_inf['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# _idx = df['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# grp_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# grp_inf_replacement


In [54]:
# # get values of _cols, where SPY_CAGR/retnStd is max after filtering out inf
# _cols = ['SPY_CAGR/retnStd']
# # _df_no_inf = df.loc[df['SPY_CAGR/retnStd'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# _idx = df['SPY_CAGR/retnStd'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# SPY_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# SPY_inf_replacement

In [55]:
# # # https://stackoverflow.com/questions/50773107/how-to-replace-infinite-value-with-maximum-value-of-a-pandas-column
# # replace inf in column grp(CAGR/UI)_mean
# df['grp(CAGR/UI)_mean'].replace(np.inf, grp_inf_replacement['grp(CAGR/UI)_mean'], inplace=True)
# # replace NaN in column grp(CAGR/UI)_std
# df['grp(CAGR/UI)_std'].replace(np.nan, grp_inf_replacement['grp(CAGR/UI)_std'], inplace=True)
# # replace NaN in column grp(CAGR/UI)_mean/std
# df['grp(CAGR/UI)_mean/std'].replace(np.nan, grp_inf_replacement['grp(CAGR/UI)_mean/std'], inplace=True)
# # replace inf in column SPY_CAGR/UI
# df['SPY_CAGR/UI'].replace(np.inf, SPY_inf_replacement, inplace=True)
# df
# df

In [56]:
# print(f'n_samples:             {n_samples:>13}')
# print(f'days_lookbacks:        {days_lookbacks}')
# print(f'days_eval:             {days_eval:>13}')
# print(f'n_top_syms:            {n_top_syms:>13}')
# print(f'syms_start:            {syms_start:>13}')
# print(f'syms_end:              {syms_end:>13}')
# print(f'grp(CAGR/UI)_mean:     {grp_CAGR_d_UI[0]:>13,.3f}')
# print(f'grp(CAGR/UI)_std:      {grp_CAGR_d_UI[1]:>13,.3f}')
# print(f'grp(CAGR/UI)_mean/std: {grp_CAGR_d_UI[2]:>13,.3f}')
# print(f'SPY_CAGR/UI:           {SPY_CAGR_d_UI[0]:>13,.3f}')